In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import mscthesis
import models
import importlib
importlib.reload(models)

<module 'models' from '/Users/kristofferek/Documents/git/mscthesis/src/py/models.py'>

In [16]:
geotweets = mscthesis.read_geotweets('sweden').set_index('userid')
tweetcount = geotweets.groupby('userid').size()
geotweets = geotweets.drop(
    labels=tweetcount[tweetcount < 5].index,
)
geotweets.head(5)

,region,createdat,tweetid,latitude,longitude,month,weekday,hourofday,timezone,ym,label,geometry
userid,,,,,,,,,,,,
5616,0,2015-05-07 15:12:52+00:00,596331871241379840,57.599221,18.436371,5,4,17,Europe/Stockholm,2015-05,other,POINT (18.43637 57.59922)
5616,0,2015-07-20 09:12:12+00:00,623057810864111616,57.599221,18.436371,7,1,11,Europe/Stockholm,2015-07,other,POINT (18.43637 57.59922)
5616,0,2015-12-23 14:43:00+00:00,679673567416565760,57.599221,18.436371,12,3,15,Europe/Stockholm,2015-12,other,POINT (18.43637 57.59922)
5616,0,2016-07-25 18:42:00+00:00,757647103011262465,57.599221,18.436371,7,1,20,Europe/Stockholm,2016-07,other,POINT (18.43637 57.59922)
5616,0,2016-08-10 06:31:05+00:00,763261365930909696,57.599221,18.436371,8,3,8,Europe/Stockholm,2016-08,other,POINT (18.43637 57.59922)


In [23]:
travel_survey_trips_mean = 3.143905 # PI? It's magic!
travel_survey_trips_std = 1.880373

model = models.PreferentialReturn(
    p=0.66, gamma=0.6, 
    region_sampling=models.RegionZipfProb(s=1.2),
    jump_size_sampling=models.JumpSizeTrueProb(),
)
sampler = models.Sampler(
    model, 
    daily_trips_sampling=models.NormalDistribution(
        travel_survey_trips_mean, 
        travel_survey_trips_std,
    ),
    n_days=7*20,
    geotweets_path="./../../dbs/sweden/geotweets.csv",
)

samples = sampler.sample(geotweets)

done with 250
done with 500
done with 750
done with 1000
done with 1250
done with 1500
done with 1750
done with 2000
done with 2250
done with 2500
done with 2750
done with 3000
done with 3250
done with 3500
done with 3750
done with 4000


In [24]:
samples.reset_index().sort_values(['userid', 'day', 'timeslot']).head(20)

,userid,day,timeslot,kind,latitude,longitude,region
0,5616,0,0,region,59.426889,17.954336,5
1,5616,0,1,region,59.405498,17.954750,6
2,5616,0,2,region,57.398813,18.874708,52
3,5616,0,3,region,59.405498,17.954750,6
4,5616,0,4,region,59.445959,17.942841,347
5,5616,1,0,region,59.426889,17.954336,5
6,5616,1,1,region,59.445959,17.942841,347
7,5616,1,2,region,59.609415,16.621037,505
8,5616,2,0,region,59.426889,17.954336,5
9,5616,2,1,region,59.619067,16.592795,371


In [25]:
samples.to_csv('./../../dbs/sweden/visits-zipf1.2.csv')